In [1]:
import requests
import csv
from bs4 import BeautifulSoup
import logging
from collections import namedtuple
import pickle
import multiprocessing
import time

from downloader import UFOData

In [2]:
dataclass = UFOData('data_agg.json')

In [4]:
dataclass.download_index_pages()

There are 492 pages found.


In [3]:
dataclass.data[2345]

{'City': 'Long Beach',
 'Date': '6/10/17 07:34',
 'Duration': '',
 'Posted': '6/15/17',
 'Shape': '',
 'State': 'CA',
 'Summary': 'At least 25 UFO floating in the sky.',
 'link': '134/S134523.html'}

In [7]:
dataclass.download_data()

In [ ]:
dataclass.save_data('xxx.json')

In [ ]:
dataclass.load_index_pages('index_pages.json')
dataclass.index_pages

In [9]:
dataclass.data[450]

{'City': 'Seattle',
 'Date': '10/8/17 02:58',
 'Duration': '3 seconds',
 'Posted': '10/19/17',
 'Shape': 'Light',
 'State': 'WA',
 'Summary': 'Very bright blue-green light moving eastward at a very high rate of speed.',
 'link': '136/S136655.html'}

In [ ]:
data, decription = tuple(item.text for item in soup.find('tbody').findAll('td'))

occurred_mark = 'Occurred'
entered_as_mark = 'Entered as'
reported_mark = 'Reported'
posted_mark = 'Posted'
location_mark = 'Location'
shape_mark = 'Shape'
duration_mark = 'Duration'

clean_data = {
    'occurred': data[data.find(occurred_mark) + len(occurred_mark):data.find(entered_as_mark) - 1].lstrip(" :").rstrip(' )'),
    'entered': data[data.find(entered_as_mark) + len(entered_as_mark):data.find(reported_mark)].lstrip(" :").rstrip(' )'),
    'reported': data[data.find(reported_mark) + len(reported_mark):data.find(posted_mark)].lstrip(" :").rstrip(' )'),
    'posted': data[data.find(posted_mark) + len(posted_mark):data.find(location_mark)].lstrip(" :").rstrip(' )'),
    'location': data[data.find(location_mark) + len(location_mark):data.find(shape_mark)].lstrip(" :").rstrip(' )'),
    'shape': data[data.find(shape_mark) + len(shape_mark):data.find(duration_mark)].lstrip(" :").rstrip(' )'),
    'duration': data[data.find(duration_mark) + len(duration_mark):].lstrip(" :").rstrip(' )'),
    #'description': decription
}
clean_data

In [ ]:
b = ' : 07/24/15 23:00)'

In [ ]:
def func(a=7):
    print(a)
func()

In [ ]:
links_to_parse = []

resp = requests.get(url=base_page + "ndxshape.html")
parsed_data = []

In [ ]:
t = time.time()

def get_data_from_table(url:str,generate_desc:bool = False):
    soup = BeautifulSoup(requests.get(url).text,"html.parser")
    header = [i.text for i in  soup.thead.findAll("th")]
    data = []
    for row in soup.tbody.findAll("tr"):
        long_desc = ""
        if generate_desc:
            try:
                long_desc =  BeautifulSoup(requests.get(base_page+row.find("a").get("href")).text,"html.parser").tbody.findAll("td")[1].text
            except Exception as e:
                logger.log(level=logging.INFO,msg="could not retrieve long description")
        i = 0
        entry = {}
        for cell in row.findAll("td"):
            entry[header[i]] = cell.text
            i+=1
        entry["Description"] = long_desc
        data.append(entry)
    return data


# Entry = namedtuple("Entry",['Date / Time', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'])

bs = BeautifulSoup(resp.text,"html.parser")
for link in bs.table.tbody.findAll("a"):
    links_to_parse.append(base_page+link.get('href'))

    
#links_to_parse = ["http://www.nuforc.org/webreports/ndxschanged.html","http://www.nuforc.org/webreports/ndxsCrescent.html","http://www.nuforc.org/webreports/ndxsCross.html","http://www.nuforc.org/webreports/ndxsDelta.html","http://www.nuforc.org/webreports/ndxsDome.html","http://www.nuforc.org/webreports/ndxsFlare.html","http://www.nuforc.org/webreports/ndxsHexagon.html","http://www.nuforc.org/webreports/ndxspyramid.html","http://www.nuforc.org/webreports/ndxsRound.html"]

for link in links_to_parse:
    data = get_data_from_table(link,False)
    parsed_data.extend(data)
    print("done: "+link +" count: "+str(len(data)))
    
print(time.time()-t)
print("checking count, should be around 113241 : "+len(parsed_data))

csv.register_dialect("semicolon",delimiter=";")
with open("big_data.csv","w",newline="",encoding="utf-8") as f:
    dw = csv.DictWriter(f,parsed_data[0].keys(),dialect="semicolon")
    dw.writeheader()
    dw.writerows(parsed_data)


In [ ]:
with open("big_data.csv","w",newline="",encoding="utf-8") as f:
    dw = csv.DictWriter(f,parsed_data[0].keys(),dialect="semicolon")
    dw.writeheader()
    for item in parsed_data :
        try:
            dw.writerow(item)
        except Exception as e:
            print(item)

In [ ]:
with open("big_save.pckl","wb") as f:
    pickle.dump(parsed_data,f)
